In [1]:
!pip install idx2numpy

  Preparing metadata (setup.py) ... done
  Created wheel for idx2numpy: filename=idx2numpy-1.2.3-py3-none-any.whl size=7903 sha256=7c253f00c6bcc246ea0830b82da1892123f148d13a041d0eacb5a429d278af5a
  Stored in directory: /root/.cache/pip/wheels/87/e5/e7/70fc742b3645ddf9d392f766feccbcc95cb3a3c806f8588af0
Successfully built idx2numpy


In [2]:
import torch
import torch.nn as nn
import pandas as pd
import idx2numpy
from torch.utils.data import Dataset , DataLoader
import torch.optim as optim
from sklearn.model_selection import train_test_split

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Load images and labels
train_images = idx2numpy.convert_from_file('/content/train-images.idx3-ubyte')
train_labels = idx2numpy.convert_from_file('/content/train-labels.idx1-ubyte')

print(train_images.shape)  # (60000, 28, 28)
print(train_labels.shape)  # (60000,)


(60000, 28, 28)
(60000,)


In [4]:
print(train_labels.dtype)

uint8


In [5]:
train_images = torch.tensor(train_images,dtype=torch.uint8)
train_labels = torch.tensor(train_labels,dtype=torch.uint8)

In [6]:
train_images

tensor([[[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        ...,

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0,

In [7]:
print(train_images.shape)
print(train_labels.shape)

torch.Size([60000, 28, 28])
torch.Size([60000])


In [8]:
train_images = train_images/255.0


In [9]:
print(train_images[0])

tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000

In [10]:
train_images = train_images.to(dtype=torch.float32)

In [11]:
train_image = train_images.reshape(60000,784).float()

In [12]:
class CustomDataset(Dataset):
  def __init__(self,features,labels):
    self.features = features
    self.labels =labels
  def __len__(self):
    return len(self.features)
  def __getitem__(self,index):
    return self.features[index],self.labels[index]

In [13]:
train_dataset = CustomDataset(train_image,train_labels)

In [14]:
len(train_dataset)

60000

In [15]:
train_dataset[1]

(tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0

In [16]:
class ANN(nn.Module):
  def __init__(self,num_features):
    super().__init__()
    self.model = nn.Sequential(
        nn.Linear(num_features,128),
        nn.ReLU(),
        nn.Linear(128,64),
        nn.ReLU(),
        nn.Linear(64,10)
    )
  def forward(self,x):
    return self.model(x)


In [17]:
train_image.shape[1]

784

In [18]:
train_loader =DataLoader(train_dataset,batch_size=32,shuffle=True)

In [19]:
learning_rate = 0.01
epochs=100

In [20]:
model = ANN(train_image.shape[1])
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr = learning_rate)

In [21]:
for epoch in range(epochs):
  total_epoch_loss=0
  for batch_features ,batch_labels in train_loader:
    batch_features,batch_labels = batch_features.to(device),batch_labels.to(device)
    output = model(batch_features)
    loss = criterion(output,batch_labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    total_epoch_loss += loss.item()
  avg = total_epoch_loss/len(train_loader)
  print(f'Epoch:{epoch+1}, Avg Loss: {avg}')

Epoch:1, Avg Loss: 1.1712932051420213
Epoch:2, Avg Loss: 0.37611061036189397
Epoch:3, Avg Loss: 0.30722306728164356
Epoch:4, Avg Loss: 0.2674477945884069
Epoch:5, Avg Loss: 0.23714570545951524
Epoch:6, Avg Loss: 0.21085033844312032
Epoch:7, Avg Loss: 0.1889626638705532
Epoch:8, Avg Loss: 0.17122381667842468
Epoch:9, Avg Loss: 0.15597195307264725
Epoch:10, Avg Loss: 0.1427946467752258
Epoch:11, Avg Loss: 0.1316393107553323
Epoch:12, Avg Loss: 0.12169865200022856
Epoch:13, Avg Loss: 0.11280590942998728
Epoch:14, Avg Loss: 0.10519048486575484
Epoch:15, Avg Loss: 0.09828370499114196
Epoch:16, Avg Loss: 0.09222748411223293
Epoch:17, Avg Loss: 0.08664624958808223
Epoch:18, Avg Loss: 0.08177268114537001
Epoch:19, Avg Loss: 0.07702444698475301
Epoch:20, Avg Loss: 0.07289312924755116
Epoch:21, Avg Loss: 0.06921124379535516
Epoch:22, Avg Loss: 0.0654816654159377
Epoch:23, Avg Loss: 0.0626555587504059
Epoch:24, Avg Loss: 0.05943821609541774
Epoch:25, Avg Loss: 0.05646496994035939
Epoch:26, Avg Lo

In [22]:
model.eval()

ANN(
  (model): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [23]:
test_images = idx2numpy.convert_from_file('/content/t10k-images.idx3-ubyte')
test_labels = idx2numpy.convert_from_file('/content/t10k-labels.idx1-ubyte')

In [25]:
test_images = torch.tensor(test_images,dtype=torch.float32)
test_labels = torch.tensor(test_labels,dtype = torch.long)
test_images

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0., 

In [27]:
test_images = test_images.reshape((10000,784))

In [28]:
test_images.shape

torch.Size([10000, 784])

In [29]:
test_dataset = CustomDataset(test_images,test_labels)

In [30]:
len(test_dataset)

10000

In [31]:
test_loader = DataLoader(test_dataset,batch_size=64,shuffle=False)

In [34]:
total = 0
correct = 0

with torch.no_grad():
  for batch_features,batch_labels in test_loader:
    batch_features,batch_labels = batch_features.to(device),batch_labels.to(device)
    outputs = model(batch_features)
    _, predicted = torch.max(outputs,1)
    total = total + batch_labels.shape[0]
    correct = correct + (predicted == batch_labels).sum().item()
print(f"Accuracy: {correct/total*100}%")

Accuracy: 96.89%
